<a href="https://colab.research.google.com/github/chilly61/OFhmMxVmv0P8XwoR/blob/main/RAG_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
hr_knowledge = [

"Human Resources professionals are responsible for recruitment, onboarding, and employee relations.",
"HR departments manage payroll, benefits administration, and compliance with labor laws.",
"Human Resources teams support performance evaluation and workforce planning.",
"HR professionals handle conflict resolution and employee engagement initiatives.",
"Talent acquisition is a core function of Human Resources.",
"HR specialists often coordinate training and professional development programs.",
"Human Resources supports organizational culture and diversity initiatives.",
"HR roles involve maintaining employee records and ensuring policy compliance.",
"Recruitment and hiring are central responsibilities in HR positions.",
"HR professionals work closely with management to support strategic staffing decisions.",
"Key skills for HR professionals include communication, interpersonal skills, and organizational ability.",

"HR roles require knowledge of employment law and workplace regulations.",
"Strong problem-solving and conflict management skills are important in HR.",
"Human Resources professionals need attention to detail and confidentiality.",
"HR positions require administrative and coordination skills.",
"People management and employee support skills are essential in HR careers.",

"Entry-level HR roles include HR Assistant, HR Coordinator, and Recruiting Intern.",
"Aspiring HR professionals often start in talent acquisition or people operations roles.",
"Internships in recruitment or employee engagement are common entry paths into HR.",
"HR assistants support senior HR managers with administrative tasks.",
"People Operations roles are closely related to Human Resources functions.",

"Job titles such as HR Generalist, HR Assistant, and Recruiter are directly related to Human Resources.",
"People Operations Specialist roles are often equivalent to HR positions.",
"Talent Acquisition Specialist is a common HR-related title.",
"Employee Relations Coordinator works within the HR function.",
"Compensation and Benefits Analyst is part of Human Resources."
]

In [3]:
!pip install sentence-transformers faiss-cpu
!pip install -q \
    transformers>=4.38.0 \
    sentence-transformers \
    faiss-cpu \
    accelerate \
    bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 90.6 MB/s eta 0:00:00


In [4]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Apziva/ProjectC.csv")
df = df.drop_duplicates(subset=['job_title'])
df.head()

Mounted at /content/drive


,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0))

model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

model.eval()
print("Generator model loaded.")


CUDA available: True
Device: Tesla T4


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Generator model loaded.


In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

doc_embeddings = embed_model.encode(hr_knowledge)
doc_embeddings = np.array(doc_embeddings).astype("float32")

index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

print("Vector index built.")
print("Total documents indexed:", len(hr_knowledge))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector index built.
Total documents indexed: 26


In [19]:
def retrieve_context(query, k=10):
    """Retrieve relevant context for a query."""
    query_embedding = embed_model.encode([query]).astype("float32")
    distances, indices = index.search(query_embedding, k)
    return [hr_knowledge[i] for i in indices[0]]

def build_scoring_prompt(job_title, target_role="Aspiring Human Resources"):
    """
Build a prompt for scoring a job title's relevance.

Args:
    job_title: The job title to score
    target_role: The target role we're matching against

Returns:
    Prompt string for the LLM
    """
    context_docs = retrieve_context(target_role, k=10)
    context = "\n".join(context_docs)

    prompt = f"""
You are an expert HR recruiter.

Target Role:
{target_role}

Relevant Background Information:
{context}

Job Title to Score:
{job_title}

Scoring Criteria (1-100):

95-100: EXACTLY what we're looking for - Students/grads studying HR, actively seeking entry-level HR roles
85-94: Currently WORKING in HR, applying HR skills
70-84: Has HR/People functions as part of broader role, or recruiting background
50-69: Adjacent roles with transferrable skills, OR management experience wanting to transition to HR
30-49: Slightly related, mostly other fields
1-29: Completely unrelated fields

Output format:
- First line: Integer score (1-100)
- Second line: One sentence explaining why

Example outputs:
95
Student actively studying and seeking entry-level HR position.
85
Working HR professional in coordinator/generalist role.
30
Director in administration, not related to HR functions.

Now score this job title:
"""
    return prompt

def generate_answer(prompt):
    """Generate answer using the LLM."""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=False,
        temperature=0.0
    )

    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]
    result = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    return result

import re

def score_job_title(job_title, target_role="Aspiring Human Resources"):
    """
Score a single job title's relevance to the target role.

Args:
    job_title: The job title to score
    target_role: The target role we're matching against

Returns:
    tuple: (score, explanation)
    """
    prompt = build_scoring_prompt(job_title, target_role)
    result = generate_answer(prompt)
    lines = result.split('\n')

    # Extract the score from the first line
    number_match = re.search(r'\b([1-9][0-9]?|100)\b', result)

    if number_match:
        score = int(number_match.group(1))
        # Ensure score is in valid range
        score = max(1, min(100, score))

        # Find where the score appears and get everything after it
        score_str = number_match.group(0)
        score_pos = result.find(score_str)
        if score_pos >= 0:
            # Get everything after the score
            after_score = result[score_pos + len(score_str):].strip()
            # Clean up common prefixes
            after_score = re.sub(r'^[\-:)\s]+', '', after_score)
            if after_score:
                explanation = after_score
            else:
                explanation = '\n'.join(lines[1:]).strip() if len(lines) > 1 else ""
        else:
            explanation = '\n'.join(lines[1:]).strip() if len(lines) > 1 else ""
    else:
        # No number found, use entire result as explanation with default score
        score = 50
        explanation = result

    explanation = '\n'.join(lines[1:]).strip() if len(lines) > 1 else ""

    return score, explanation

In [20]:
# Test the scoring function
test_titles = [
    "HR Assistant",
    "Human Resource Assistant",
    "People Operations Intern",
    "Marketing Data Analyst",
    "Software Engineer"
]

print("Testing scoring function:")
print("=" * 60)

for title in test_titles:
    score, explanation = score_job_title(title)
    print(f"\n{title}")
    print(f"Score: {score}")
    print(f"Explanation: {explanation}")
    print("-" * 40)

Testing scoring function:

HR Assistant
Score: 95
Explanation: 95
Student actively studying and seeking entry-level HR position. The role of an HR Assistant aligns perfectly with the target role of Aspiring Human Resources, making it an ideal fit for someone who is either currently studying or actively seeking
----------------------------------------

Human Resource Assistant
Score: 95
Explanation: 95
Student actively studying and seeking entry-level HR position. The role of a Human Resource Assistant aligns closely with entry-level HR positions, making it an excellent fit for aspiring HR professionals who are either currently studying or actively seeking
----------------------------------------

People Operations Intern
Score: 95
Explanation: 95
This role is closely aligned with the core functions of Human Resources, including talent acquisition and employee support, making it an excellent fit for aspiring HR professionals.
----------------------------------------

Marketing Data Anal

In [21]:
def score_and_sort(df, title_col="job_title", target_role="Aspiring Human Resources"):
    """
Score all job titles and sort by relevance.

This replaces the pairwise comparison approach with direct scoring (1-100).

Args:
    df: DataFrame containing job titles
    title_col: Column name for job titles
    target_role: The target role we're matching against

Returns:
    DataFrame sorted by score in descending order
    """
    titles = df[title_col].tolist()
    results = []

    total = len(titles)
    for idx, title in enumerate(titles):
        print(f"Scoring {idx+1}/{total}: {title}")

        score, explanation = score_job_title(title, target_role)

        results.append({
            "job_title": title,
            "score": score,
            "explanation": explanation
        })

    # Create DataFrame and sort by score descending
    scored_df = pd.DataFrame(results)
    scored_df = scored_df.sort_values(by="score", ascending=False).reset_index(drop=True)

    return scored_df

# Score and sort all job titles
scored_df = score_and_sort(df, title_col="job_title", target_role="Aspiring Human Resources")

print("\n" + "=" * 60)
print("Final Ranking (sorted by score 1-100):")
print("=" * 60)
print(scored_df.to_string())

Scoring 1/52: 2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional
Scoring 2/52: Native English Teacher at EPIK (English Program in Korea)
Scoring 3/52: Aspiring Human Resources Professional
Scoring 4/52: People Development Coordinator at Ryan
Scoring 5/52: Advisory Board Member at Celal Bayar University
Scoring 6/52: Aspiring Human Resources Specialist
Scoring 7/52: Student at Humber College and Aspiring Human Resources Generalist
Scoring 8/52: HR Senior Specialist
Scoring 9/52: Seeking Human Resources HRIS and Generalist Positions
Scoring 10/52: Student at Chapman University
Scoring 11/52: SVP, CHRO, Marketing & Communications, CSR Officer | ENGIE | Houston | The Woodlands | Energy | GPHR | SPHR
Scoring 12/52: Human Resources Coordinator at InterContinental Buckhead Atlanta
Scoring 13/52: Aspiring Human Resources Management student seeking an internship
Scoring 14/52: Seeking Human Resources Opportunities
Scoring 15/52: Experienced 

In [18]:
# Save results to CSV
output_path = "/content/drive/MyDrive/Apziva/RAG_scored_job_titles.csv"
scored_df.to_csv(output_path, index=False)
print(f"Results saved to: {output_path}")

# Display top and bottom results
print("\n" + "=" * 60)
print("TOP 10 Most Relevant to 'Aspiring Human Resources':")
print("=" * 60)
print(scored_df.head(10).to_string(index=False))

print("\n" + "=" * 60)
print("BOTTOM 5 Least Relevant:")
print("=" * 60)
print(scored_df.tail(5).to_string(index=False))

Results saved to: /content/drive/MyDrive/Apziva/RAG_scored_job_titles.csv

TOP 10 Most Relevant to 'Aspiring Human Resources':
                                                                                                  job_title  score explanation
                                                                                                    Student     95          95
   2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional     50            
                                                                      Aspiring Human Resources Professional     50            
                                                  Native English Teacher at EPIK (English Program in Korea)     50            
                                                            Advisory Board Member at Celal Bayar University     50            
                                                                        Aspiring Human Resources Specialist    